In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
import warnings
warnings.filterwarnings("ignore")

# Load data
ground_truth = pd.read_csv("../source_data/test_preprocess.csv")
predictions = {
    "SVM": pd.read_csv('../evaluation/svm_predictions.csv'),
    "BERT": pd.read_csv('../evaluation/bert_predictions.csv'),
    # zero-shot models
    "GPT-3.5 Turbo Zero-Shot": pd.read_csv('../evaluation/zero_shot_gpt-3.5-turbo_predictions.csv'),
    "GPT-4 Turbo Zero-Shot": pd.read_csv('../evaluation/zero_shot_gpt-4-turbo_predictions.csv'),
    "GPT-4o Zero-Shot": pd.read_csv('../evaluation/zero_shot_gpt-4o_predictions.csv'),
    # few-shot models
    "GPT-3.5 Turbo Few-Shot": pd.read_csv('../evaluation/few_shot_gpt-3.5-turbo_predictions.csv'),
    "GPT-4 Turbo Few-Shot": pd.read_csv('../evaluation/few_shot_gpt-4-turbo_predictions.csv'),
    "GPT-4o Few-Shot": pd.read_csv('../evaluation/few_shot_gpt-4o_predictions.csv'),
    
}

# Configuration
aspect_keys = ["ac", "air_panas", "bau", "general", "kebersihan", 
               "linen", "service", "sunrise_meal", "tv", "wifi"]

def evaluate_model(true_df, pred_df, model_name):
    """Core evaluation function for a single model"""
    merged = true_df.merge(pred_df, on="review", suffixes=('_true', '_pred'))
    results = []
    
    for aspect in aspect_keys:
        y_true = merged[f"{aspect}_true"]
        y_pred = merged[f"{aspect}_pred"]
        
        report = classification_report(y_true, y_pred, 
                                      target_names=['neg', 'pos', 'neut'],
                                      output_dict=True)
        
        results.append({
            'model': model_name,
            'aspect': aspect,
            'accuracy': accuracy_score(y_true, y_pred),
            'macro_precision': report['macro avg']['precision'],
            'macro_recall': report['macro avg']['recall'],
            'macro_f1': report['macro avg']['f1-score'],
            'weighted_precision': report['weighted avg']['precision'],
            'weighted_recall': report['weighted avg']['recall'],
            'weighted_f1': report['weighted avg']['f1-score'],
            'neg_precision': report['neg']['precision'],
            'neg_recall': report['neg']['recall'],
            'neg_f1': report['neg']['f1-score'],
            'pos_precision': report['pos']['precision'],
            'pos_recall': report['pos']['recall'],
            'pos_f1': report['pos']['f1-score'],
            'neut_precision': report['neut']['precision'],
            'neut_recall': report['neut']['recall'],
            'neut_f1': report['neut']['f1-score'],
            'support': report['weighted avg']['support']
        })
    
    return pd.DataFrame(results)

# Main evaluation pipeline
all_results = []
for model_name, pred_df in predictions.items():
    print(f"Evaluating {model_name}...")
    model_results = evaluate_model(ground_truth, pred_df, model_name)
    all_results.append(model_results)

# Generate key outputs
full_results = pd.concat(all_results, ignore_index=True)

Evaluating SVM...
Evaluating BERT...
Evaluating GPT-3.5 Turbo Few-Shot...
Evaluating GPT-4 Turbo Few-Shot...
Evaluating GPT-4o Few-Shot...


In [14]:
# 1. Model Comparison Table
model_comparison = full_results.groupby('model').agg({
    'accuracy': 'mean',
    'macro_precision': 'mean',
    'macro_recall': 'mean',
    'macro_f1': 'mean',
    'weighted_precision': 'mean',
    'weighted_recall': 'mean',
    'weighted_f1': 'mean',
    'neg_precision': 'mean',
    'neg_recall': 'mean',
    'neg_f1': 'mean',
    'pos_precision': 'mean',
    'pos_recall': 'mean',
    'pos_f1': 'mean',
    'neut_precision': 'mean',
    'neut_recall': 'mean',
    'neut_f1': 'mean',
    'support': 'mean'
}).reset_index()

print("\n=== AVG Model Comparison ===")
model_comparison.round(4)


=== AVG Model Comparison ===


,model,accuracy,macro_precision,macro_recall,macro_f1,weighted_precision,weighted_recall,weighted_f1,neg_precision,neg_recall,neg_f1,pos_precision,pos_recall,pos_f1,neut_precision,neut_recall,neut_f1,support
0,BERT,0.9608,0.8129,0.8093,0.8060,0.9595,0.9608,0.9595,0.8238,0.8362,0.8269,0.9787,0.9761,0.9774,0.6363,0.6155,0.6136,286.0
1,GPT-3.5 Turbo Few-Shot,0.9154,0.8143,0.8193,0.8058,0.9221,0.9154,0.9160,0.7962,0.7544,0.7505,0.9399,0.9559,0.9469,0.7068,0.7475,0.7200,286.0
2,GPT-4 Turbo Few-Shot,0.9311,0.8381,0.8613,0.8306,0.9484,0.9311,0.9343,0.8362,0.8156,0.8031,0.9609,0.9533,0.9544,0.7171,0.8150,0.7345,286.0
3,GPT-4o Few-Shot,0.9455,0.8424,0.8603,0.8391,0.9513,0.9455,0.9455,0.8549,0.8148,0.8220,0.9625,0.9703,0.9654,0.7099,0.7959,0.7299,286.0
4,SVM,0.9350,0.7000,0.6718,0.6724,0.9293,0.9350,0.9305,0.7453,0.7450,0.7435,0.9583,0.9741,0.9660,0.3962,0.2965,0.3077,286.0


In [15]:
# 2. Aspect-Level Performance
aspect_performance = full_results.groupby(['model', 'aspect']).agg({
    'macro_precision': 'mean',
    'macro_recall': 'mean',
    'macro_f1': 'mean',
    'weighted_precision': 'mean',
    'weighted_recall': 'mean',
    'weighted_f1': 'mean',
    'neg_precision': 'mean',
    'neg_recall': 'mean',
    'neg_f1': 'mean',
    'pos_precision': 'mean',
    'pos_recall': 'mean',
    'pos_f1': 'mean',
    'neut_precision': 'mean',
    'neut_recall': 'mean',
    'neut_f1': 'mean',
    'support': 'mean'
}).reset_index()

print("\n=== AVG Aspect-Level Performance ===")
aspect_performance.round(4)


=== AVG Aspect-Level Performance ===


,model,aspect,macro_precision,macro_recall,macro_f1,weighted_precision,weighted_recall,weighted_f1,neg_precision,neg_recall,neg_f1,pos_precision,pos_recall,pos_f1,neut_precision,neut_recall,neut_f1,support
0,BERT,ac,0.9258,0.9838,0.9510,0.9908,0.9895,0.9898,0.9773,0.9556,0.9663,1.0000,0.9957,0.9978,0.8000,1.0000,0.8889,286.0
1,BERT,air_panas,0.6084,0.6349,0.6209,0.9603,0.9650,0.9624,0.8333,0.9211,0.8750,0.9918,0.9837,0.9877,0.0000,0.0000,0.0000,286.0
2,BERT,bau,0.9378,0.9647,0.9503,0.9644,0.9615,0.9624,0.8261,0.9268,0.8736,0.9874,0.9672,0.9772,1.0000,1.0000,1.0000,286.0
3,BERT,general,0.7168,0.6873,0.6963,0.9404,0.9441,0.9419,0.3333,0.2000,0.2500,0.9671,0.9671,0.9671,0.8500,0.8947,0.8718,286.0
4,BERT,kebersihan,0.8854,0.9170,0.8996,0.9069,0.9056,0.9057,0.8830,0.8646,0.8737,0.9355,0.9177,0.9265,0.8378,0.9688,0.8986,286.0
5,BERT,linen,0.9427,0.7696,0.8205,0.9314,0.9301,0.9273,0.8780,0.9000,0.8889,0.9500,0.9645,0.9572,1.0000,0.4444,0.6154,286.0
6,BERT,service,0.9482,0.9300,0.9389,0.9575,0.9580,0.9577,0.8780,0.8372,0.8571,0.9667,0.9807,0.9736,1.0000,0.9722,0.9859,286.0
7,BERT,sunrise_meal,0.9045,0.8856,0.8947,0.9820,0.9825,0.9822,0.8462,0.7857,0.8148,0.9925,0.9962,0.9943,0.8750,0.8750,0.8750,286.0
8,BERT,tv,0.6129,0.6641,0.6361,0.9748,0.9825,0.9780,0.8387,1.0000,0.9123,1.0000,0.9922,0.9961,0.0000,0.0000,0.0000,286.0
9,BERT,wifi,0.6468,0.6558,0.6512,0.9862,0.9895,0.9878,0.9444,0.9714,0.9577,0.9960,0.9960,0.9960,0.0000,0.0000,0.0000,286.0
